In [9]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import random as rn

In [3]:
tf.__version__

'2.2.0'

In [4]:
tf.keras.backend.clear_session()
df=pd.read_csv("labels_final.csv")
df.head(5)

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [5]:
df['label'] = df['label'].apply(str)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(validation_split=0.3,rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=df,directory="./rvl-cdip/data_final/",x_col="path"
                                           ,y_col="label",subset="training"
                                            ,batch_size=64,seed=42,shuffle=True,validate_filenames=False
                                            ,class_mode="categorical"
                                            ,target_size=(224,224))

Found 33600 non-validated image filenames belonging to 16 classes.


In [7]:
test_generator=datagen.flow_from_dataframe(dataframe=df,directory="./rvl-cdip/data_final/",x_col="path"
                                            ,y_col='label',subset="validation"
                                            ,batch_size=64,seed=42,shuffle=True
                                            ,class_mode="categorical"
                                            ,target_size=(224,224))

Found 14400 validated image filenames belonging to 16 classes.


In [7]:
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
tf.keras.backend.clear_session()
np.random.seed(0)
rn.seed(0)
new_input = Input(shape=(256, 256,3))
input_layer = Input(shape=(256,256,3),name = 'Input_Layer')

model_vgg16 = VGG16(weights='imagenet', include_top=False,input_tensor=input_layer)
for layer in model_vgg16.layers:
	layer.trainable = False
output_vgg16 = model_vgg16(input_layer)

x= Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='x')(output_vgg16)
Pool1 = MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',name='Pool1')(x)
flatten = Flatten(data_format='channels_last',name='Flatten')(Pool1)

FC1 = Dense(units=64,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='FC1')(flatten)
FC2 = Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=33),name='FC2')(FC1)
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(FC2)
model=Model(inputs=input_layer,outputs=Out)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Layer (InputLayer)     [(None, 256, 256, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
x (Conv2D)                   (None, 6, 6, 32)          147488    
_________________________________________________________________
Pool1 (MaxPooling2D)         (None, 3, 3, 32)          0         
_________________________________________________________________
Flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 64)                18496     
_________________________________________________________________
FC2 (Dense)                  (None, 32)                2080  

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss="categorical_crossentropy",metrics=['accuracy'])

In [ ]:
model.fit(train_generator,steps_per_epoch=300,validation_data=test_generator,validation_steps=225,epochs=3)

Epoch 1/3
300/300 [==============================] - 6622s 22s/step - loss: 2.0765 - accuracy: 0.3236 - val_loss: 1.7455 - val_accuracy: 0.4395
Epoch 2/3
102/300 [=========>....................] - ETA: 53:59 - loss: 1.7217 - accuracy: 0.4436

In [11]:
tf.keras.backend.clear_session()
np.random.seed(0)
rn.seed(0)
input_layer = Input(shape=(224,224,3),name = 'Input_Layer')

model_vgg16 = VGG16(weights='imagenet', include_top=False,input_tensor=input_layer)
for layer in model_vgg16.layers:
    layer.trainable = False
output_vgg16 = model_vgg16(input_layer)
x=Conv2D(filters=4096,kernel_size=(7,7),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='x')(output_vgg16)
y=Conv2D(filters=16,kernel_size=(1,1),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='y')(x)
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(y)
model=Model(inputs=input_layer,outputs=Out)

In [1]:
from keras.layers import Input, Flatten, Dense,GlobalMaxPooling2D
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense
from keras.models import Model
input_layer = Input(shape=(224,224,3),name = 'Input_Layer')

model_vgg16 = VGG16(weights='imagenet', include_top=False,input_tensor=input_layer)
for layer in model_vgg16.layers:
    layer.trainable = False
for layer in model_vgg16.layers[-6:]:
    layer.trainable = True
output_vgg16 = model_vgg16(input_layer)
x=Conv2D(filters=4096,kernel_size=(7,7),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='x')(output_vgg16)
y=Conv2D(filters=16,kernel_size=(1,1),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='y')(x)
O=GlobalMaxPooling2D(data_format='channels_last')(y)
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(O)

model=Model(inputs=input_layer,outputs=Out)

NameError: name 'tf' is not defined

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
x (Conv2D)                   (None, 1, 1, 4096)        102764544 
_________________________________________________________________
y (Conv2D)                   (None, 1, 1, 16)          65552     
_________________________________________________________________
Output (Dense)               (None, 1, 1, 16)          272       
Total params: 117,545,056
Trainable params: 102,830,368
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss="categorical_crossentropy",metrics=['accuracy'])

In [1]:
model.fit(train_generator,steps_per_epoch=300,validation_data=test_generator,validation_steps=225,epochs=3)

NameError: name 'model' is not defined